In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
from scipy.ndimage import gaussian_filter, map_coordinates 
from typing import Any
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR


In [9]:
df = pd.read_csv('pages.csv')
df_1 = pd.DataFrame(df).rename(columns={'Unnamed: 1': 'fonts'})
df_2 = df_1['pages'].str.replace('^img\\\\', '', regex=True)
df_rete = df_1[['pages', 'fonts']].copy()
df_rete['pages'] = df_rete['pages'].str.replace('^img\\\\', '', regex=True)

In [10]:
current_path = os.getcwd()
# Define the folder path where the images are stored
image_folder = f"{current_path}\\TheLibrarianFromAlexandria\\img"
output_folder = f"{current_path}\\TheLibrarianFromAlexandria\\immagini_formattate"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_folder_2 = f"{current_path}\\TheLibrarianFromAlexandria\\immagini_reteneurale"
if not os.path.exists(output_folder_2):
    os.makedirs(output_folder_2)

In [ ]:
# Funzione di pre-processing e salvataggio dell'immagine finale (denoised)
def preprocess_and_save_final_image(img: Any, img_name: str):
    # **1. Conversione in scala di grigi**
    gray = img.convert("L")  # 'L' = grayscale
    # **2. Binarizzazione**
    threshold = 128  # Soglia per la binarizzazione
    binary = gray.point(lambda p: 255 if p > threshold else 0)
    # **3. Rimozione del Rumore (Denoising)**
    denoised = binary.filter(ImageFilter.MedianFilter(size=3))  # Rimozione del rumore tramite filtro mediano
    # Salva solo l'immagine denoised
    denoised_image_path = os.path.join(output_folder, f"{img_name}")
    denoised.save(denoised_image_path)  # Salva l'immagine denoised
    print(f"Immagine denoised salvata come: {denoised_image_path}")

In [ ]:
# Funzione di Data Augmentation
def data_augmentation(image):
    # **1. Trasformazioni base con torchvision**
    augmentation = transforms.Compose([
                      # Rotazione casuale tra -20 e 20 gradi
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Traslazione del 10%
        transforms.RandomHorizontalFlip(p=0.5),              # Ribaltamento orizzontale
        transforms.RandomVerticalFlip(p=0.3),                # Ribaltamento verticale
        transforms.ColorJitter(brightness=0.3, contrast=0.3), # Variazione luminosità e contrasto
    ])
    augmented_image = augmentation(image)  # Applica le trasformazioni base
    return augmented_image  # Restituisce l'immagine trasformata

# Funzione per aggiungere rumore
def add_noise(image):
    arr = np.array(image).astype(np.float32)
    noise = np.random.normal(0, 25, arr.shape)
    noisy = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(noisy)

# Funzione per distorsione elastica
def elastic_transform(image, alpha=1000, sigma=40):
    arr = np.array(image)
    shape = arr.shape
    dx = gaussian_filter((np.random.rand(*shape)*2 - 1), sigma) * alpha
    dy = gaussian_filter((np.random.rand(*shape)*2 - 1), sigma) * alpha
    x, y = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]))
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1))
    distorted = map_coordinates(arr, indices, order=1, mode='reflect').reshape(shape)
    return Image.fromarray(distorted.astype(np.uint8))

# Funzione che applica tutte le trasformazioni in sequenza
def apply_all_transformations(image, img_name):
    # 1. Applica Data Augmentation
    augmented_image = data_augmentation(image)
    # 2. Aggiungi rumore
    noisy_image = add_noise(augmented_image)
    # 3. Applica distorsione elastica
    final_image = elastic_transform(noisy_image)
    # Salva l'immagine finale
    final_image_path = os.path.join(output_folder_2, f"{img_name}")
    final_image.save(final_image_path)  # Salva l'immagine finale
    print(f"Immagine finale salvata come: {final_image_path}")

In [ ]:
# Itera sulla colonna 'pages' del DataFrame
for i in range(len(df)):  # Limita la visualizzazione a 5 immagini
    try:
        img_name = df_2.iloc[i] # Ottieni il nome del file dal DataFrame
        img_path = os.path.join(image_folder, img_name)
        # Verifica se il file esiste
        if not os.path.exists(img_path):
            print(f"Immagine {img_name} non trovata.")
            continue
        
        # Carica l'immagine
        img = Image.open(img_path)
        # Esegui il Data Augmentation e salva le immagini
        print(f"\nAnalisi immagine {i + 1}: {img_name}")
        preprocess_and_save_final_image(img, img_name)
        
    except Exception as e:
        print(f"Errore nell'aprire l'immagine {img_name}: {e}")

# Itera sulla colonna 'pages' del DataFrame
for i in range(len(df)):  # Limita la visualizzazione a 5 immagini
    try:
        img_name_2 = df_2.iloc[i]  # Ottieni il nome del file dal DataFrame
        img_path_2 = os.path.join(output_folder, img_name_2)
        # Verifica se il file esiste
        if not os.path.exists(img_path_2):
            print(f"Immagine {img_name_2} non trovata.")
            continue
        # Carica l'immagine
        img_2 = Image.open(img_path_2)
        # Esegui il pre-processing e salva solo l'ultima versione (denoised)
        print(f"\nAnalisi immagine {i + 1}: {img_name_2}")
        apply_all_transformations(img_2, img_name_2)
    except Exception as e:
        print(f"Errore nell'aprire l'immagine {img_name_2}: {e}")

Convutional Neural Network (ResNet40--> Pretrained)

In [11]:
image_folder_rete = f'{current_path}\\TheLibrarianFromAlexandria\\immagini_reteneurale'
image_names = os.listdir(image_folder_rete)  
fonts = list(df_rete['fonts'].unique())
train_df, test_df = train_test_split(df_rete, test_size=0.2, random_state=42)
print(f"Training set: {len(train_df)} immagini")
print(f"Test set: {len(test_df)} immagini")

Training set: 1004 immagini
Test set: 252 immagini


In [12]:
# Step 3: Definisci le trasformazioni per il preprocessing
transform = transforms.Compose([
    transforms.Resize(256),            # Ridimensiona le immagini a 256x256
    transforms.CenterCrop(224),        # Ritaglia il centro dell'immagine a 224x224
    transforms.ToTensor(),             # Converte l'immagine in un tensor PyTorch
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizza con la media e la deviazione standard di ImageNet
])

class CustomDataset(Dataset):
    def __init__(self, dataframe, image_folder_rete, transform=None):
        self.dataframe = dataframe
        self.image_folder_rete = image_folder_rete
        self.transform = transform

                # Mappa dei font (assicurati che corrisponda ai font nel tuo dataset)
        self.font_map = {"vesta": 0, "aureus": 1, "roman": 2, "cicero": 3, "colosseum": 4,
                         "augustus": 5, "consul": 6, "laurel": 7, "forum": 8, "trajan": 9, "senatus": 10}
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder_rete, self.dataframe.iloc[idx, 0])  # Ottieni il nome del file dell'immagine
        image = Image.open(img_name).convert('RGB') 
        label = self.dataframe.iloc[idx, 1]  # La colonna 'labels' contiene le etichette

        if self.transform:
            image = self.transform(image)
            
        # Converte il label da stringa a numero usando la mappa
        label = self.font_map[label]
        # Restituisci i dati come una tupla di tensori (pagine, etichette)
        # La variabile `label` dovrebbe essere un singolo tensore, non una tupla
        return image, torch.tensor(label)  # Assicurati che il label sia un tensore PyTorch

In [ ]:
# Step 5: Crea il DataLoader per Training e Test
train_dataset = CustomDataset(train_df, image_folder_rete, transform)
test_dataset = CustomDataset(test_df, image_folder_rete, transform)
batch_size=35
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Riduci il batch_size per tenere conto del numero ridotto di immagini
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 6: Carica il modello pre-addestrato (ResNet50)
model = models.resnet50(pretrained=True)
# Congela i parametri del modello pre-addestrato
for param in model.parameters():
    param.requires_grad = False
# Aggiungi un nuovo classificatore (adatta al numero di classi del tuo dataset)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(df_rete['fonts'].unique()))  # Cambia in base al numero di classi
# Sposta il modello sulla GPU se disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# Step 7: Definisci la funzione di ottimizzazione e loss
criterion = nn.CrossEntropyLoss()  # Funzione di perdita per la classificazione multi-classe
optimizer = optim.SGD(model.fc.parameters(), lr=0.08, momentum=0.9)

In [ ]:
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

# Variabili per memorizzare i risultati durante l'addestramento
train_losses = []
train_accuracies = []
test_accuracies = []

# Aggiungi l'ottimizzatore (SGD con momentum)
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# Aggiungi un Learning Rate Scheduler (StepLR)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Ogni 5 epoche, riduci il learning rate di un fattore 0.1

# Early stopping variabili
best_accuracy = 0.0
patience = 5  # Numero di epoche senza miglioramento prima di fermare l'addestramento
epochs_without_improvement = 0

# Step 8: Addestra il modello (per 15 epoche)
for epoch in range(15):  # Esegui per 15 epoche
    model.train()  # Imposta il modello in modalità training
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Ciclo di addestramento
    for pages, fonts in train_loader:
        pages, fonts = pages.to(device), fonts.to(device)

        optimizer.zero_grad()
        outputs = model(pages)
        loss = criterion(outputs, fonts)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += fonts.size(0)
        correct += (predicted == fonts).sum().item()

    # Calcolo della loss e accuracy per il training set
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total * 100
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f'Epoch {epoch+1}, Training Loss: {train_loss}, Training Accuracy: {train_accuracy}%')

    # Step 9: Testa il modello sul test set
    model.eval()  # Imposta il modello in modalità valutazione
    correct = 0
    total = 0
    with torch.no_grad():  # Disabilita il calcolo dei gradienti per il test
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total * 100
    test_accuracies.append(test_accuracy)
    print(f'Test Accuracy: {test_accuracy}%')

    # Early Stopping: verifica se la performance è migliorata
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    # Fermiamo l'addestramento se non ci sono miglioramenti per 'patience' epoche
    if epochs_without_improvement >= patience:
        print("Early stopping")
        break

    # Aggiorna il learning rate con il scheduler
    scheduler.step()

# Salva i risultati in un file Excel o CSV per analisi future (facoltativo)
import pandas as pd

results = {
    'Epoch': list(range(1, len(train_losses) + 1)),
    'Training Loss': train_losses,
    'Training Accuracy': train_accuracies,
    'Test Accuracy': test_accuracies
}

df_results = pd.DataFrame(results)

# Salva in un file CSV o Excel
df_results.to_excel('model_results.xlsx', index=False)
print("Risultati salvati in 'model_results.xlsx'")

# Visualizza i grafici della Loss e Accuracy
plt.figure(figsize=(12, 6))

# Grafico della Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss per Epoca')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()

# Grafico della Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Accuracy per Epoca')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [15]:
# Variabili per memorizzare i risultati durante l'addestramento
train_losses = []
train_accuracies = []
test_accuracies = []

# Step 8: Addestra il modello (solo per 1 epoca dato il numero ridotto di immagini)
for epoch in range(15):  # Esegui più epoche se necessario
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for pages, fonts in train_loader:
        pages, fonts = pages.to(device), fonts.to(device)

        optimizer.zero_grad()
        outputs = model(pages)
        loss = criterion(outputs, fonts)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += fonts.size(0)
        correct += (predicted == fonts).sum().item()

    # Calcolo della loss e accuracy per il training set
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total * 100
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f'Epoch {epoch+1}, Training Loss: {train_loss}, Training Accuracy: {train_accuracy}%')

    # Step 9: Testa il modello sul test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_accuracy = correct / total * 100
    test_accuracies.append(test_accuracy)
    print(f'Test Accuracy: {test_accuracy}%')

c:\Users\corra\OneDrive\Desktop\tests\23-mlproj\venv\Lib\site-packages\PIL\Image.py:3442: DecompressionBombWarning: Image size (103857120 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\corra\OneDrive\Desktop\tests\23-mlproj\venv\Lib\site-packages\PIL\Image.py:3442: DecompressionBombWarning: Image size (103803320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\corra\OneDrive\Desktop\tests\23-mlproj\venv\Lib\site-packages\PIL\Image.py:3442: DecompressionBombWarning: Image size (103803147 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\corra\OneDrive\Desktop\tests\23-mlproj\venv\Lib\site-packages\PIL\Image.py:3442: DecompressionBombWarning: Image size (103811050 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\corra\OneDrive\Desktop\tests\23-mlproj\venv

Epoch 1, Training Loss: 2.303737681487511, Training Accuracy: 16.83266932270916%
Test Accuracy: 23.41269841269841%
Epoch 2, Training Loss: 1.8791441218606357, Training Accuracy: 39.243027888446214%
Test Accuracy: 39.682539682539684%
Epoch 3, Training Loss: 1.5868562048879162, Training Accuracy: 55.079681274900395%
Test Accuracy: 46.82539682539682%
Epoch 4, Training Loss: 1.449908815581223, Training Accuracy: 56.37450199203188%
Test Accuracy: 47.61904761904761%
Epoch 5, Training Loss: 1.3155281749264947, Training Accuracy: 63.645418326693225%
Test Accuracy: 48.01587301587302%
Epoch 6, Training Loss: 1.2616627935705513, Training Accuracy: 61.952191235059765%
Test Accuracy: 49.60317460317461%
Epoch 7, Training Loss: 1.1825131403988804, Training Accuracy: 66.23505976095618%
Test Accuracy: 54.36507936507936%
Epoch 8, Training Loss: 1.1449312094984383, Training Accuracy: 66.03585657370517%
Test Accuracy: 55.55555555555556%
Epoch 9, Training Loss: 1.085456367196708, Training Accuracy: 68.8247

In [ ]:
# Definisci il percorso per la cartella 'model_charts'
model_result_path = 'C:\\Users\\corra\\OneDrive\\desktop\\tests\\23-mlproj\\model_results'
# Crea la cartella se non esiste
if not os.path.exists(model_result_path):
    os.makedirs(model_result_path)
# Crea i grafici
plt.figure(figsize=(10, 5))
# Grafico della perdita (loss)
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss per Epoca')
plt.xlabel('Epoche')
plt.ylabel('Loss')
plt.legend()
# Grafico della precisione (accuracy)
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Accuracy per Epoca')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()
# Salva i grafici nella cartella 'model_charts'
plt.tight_layout()
plt.savefig(os.path.join(model_result_path, f'training_and_accuracy_charts_{epoch+1}_{batch_size}.png'))
# Mostra il grafico
plt.show()

In [ ]:
# Crea un DataFrame con i risultati
results = {
    'Epoch': list(range(1, len(train_losses) + 1)),
    'Training Loss': train_losses,
    'Training Accuracy': train_accuracies,
    'Test Accuracy': test_accuracies
}
df_results = pd.DataFrame(results)
# Salva i risultati in un file CSV
df_results.to_excel(f'model_results\\training_results_{epoch+1}_{batch_size}.xlsx', index=False)
print("Risultati esportati nel file 'training_results.csv'")